### Prepare the datasets from text files-cramp database

In [ ]:
import pandas as pd


### only want the name and sequence 

In [ ]:
with open("Antibacterial_exp2.txt", 'r', encoding='latin-1') as arquivo:
    conteudo = arquivo.read()

linhas = conteudo.split('\n')
linhas_filtradas = [linha for linha in linhas if linha.startswith('DRAMP')]

valores_campos = []
for linha in linhas_filtradas:
    campos = linha.split('\t')
    valor_campo_2 = campos[3] if len(campos) > 3 else None
    valor_campo_3 = campos[1] if len(campos) > 1 else None
    valores_campos.append((valor_campo_2, valor_campo_3))
valores_campos

In [ ]:
#to remove the first tupple - its not necessary
valores_campos.pop(0)
len(valores_campos)
print(valores_campos)

### maintain the negative class because we only add new peptides relacionated with positiva class

In [ ]:
df = pd.read_csv('peptides-all.csv',encoding="latin-1")

df = df[df['Type'] != 1]

print(df)

## make the features based on the protein sequence of each peptide

In [ ]:
from propy import PyPro
from propy.GetProteinFromUniprot import GetProteinSequence
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import urllib.error

from urllib.parse import quote

all_descriptors = []

for tupple in valores_campos:
    DesObject = PyPro.GetProDes(tupple[1])
    
    X = ProteinAnalysis(tupple[1])
    descriptors = [
        tupple[0],
        1,
        tupple[1],
        X.length,
        X.aromaticity(),
        X.isoelectric_point(),
        DesObject.GetAAComp(),
        DesObject.GetDPComp(),
        DesObject.GetCTD()
        ]
    try:
        descriptors.insert(4, X.molecular_weight())
        all_descriptors.append(descriptors)
    except ValueError as e:
        print(f"Ignorando erro de valor para o cálculo de peso molecular da tupla {tupple}: {e}")
            
            
    

In [ ]:
len(all_descriptors)


### put the info in a list

In [ ]:
def extract_values(data):
    values = []
    if isinstance(data, list):
        for item in data:
            values.extend(extract_values(item))
    elif isinstance(data, dict):
        for value in data.values():
            values.extend(extract_values(value))
    else:
        values.append(data)
    return values



nova_data=[]
for x in all_descriptors:

    values = extract_values(x)
    nova_data.append(values)
    print(values)




In [ ]:
#the ones that does not have id
indices = [i for i, item in enumerate(nova_data) if item[0] == '']
len(indices)


### Merge the dataset with negative part and positive part

In [ ]:
siga=nova_data #for security

In [ ]:
df2=pd.DataFrame(siga)
df2.shape

In [ ]:
df.shape

In [ ]:
siga_columns = df.columns

dfnovo = pd.DataFrame(siga, columns=siga_columns)

df_concatenated = pd.concat([df, dfnovo], ignore_index=True)

print(df_concatenated.tail(3))


In [ ]:
# dataset done for the first approach
df_concatenated.to_csv('meus_peptides.csv', index=False)
